## Tratamento dos dados

### Importando bibliotecas

In [1]:
import numpy as np
from sklearn.metrics import adjusted_rand_score
import pandas as pd
import seaborn as sns
from sklearn.metrics import pairwise_distances

### Carregando o dataset

In [2]:
df = sns.load_dataset('iris')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


### Verificando as classes

In [3]:
df["species"].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

### Verificando o nome exato das colunas

In [4]:
df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

### Transformando classes em números

In [5]:
df["species"].replace({"setosa": 0, "versicolor": 1, "virginica": 2}, inplace=True)
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### Verificando a corretude das classes

In [6]:
df["species"].unique()

array([0, 1, 2], dtype=int64)

### Renomeando a coluna da classe

In [7]:
df.columns = ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm", "Class"]
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### Armazenando as classes em uma variável separada

In [8]:
labels = df["Class"].values
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

### Isolando os dados da classe

In [9]:
df.drop("Class", axis=1, inplace=True)
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


### Transformando os dados em uma array

In [10]:
dados = df.to_numpy()
dados

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

## Clustering

### Inicialização da matriz de pertinência

A matriz de pertinência é inicializada aleatoriamente $u_{ik}(i=1,...c$ e $k=1,...,n)$ do objeto $k$ pertencente ao grupo $C_i$ tal que:
- $u_{ik} \in [0,1]$;
- $0 < \sum_{k=1}^nu_{ik} < n$;
- $\sum_{i=1}^cu_{ik}=1$ para todo $k \in \Omega$.

In [11]:
def inicializacao_matriz_pertinencia(num_amostras, num_clusters):
    matriz_pertinencia = np.random.rand(num_amostras, num_clusters) # gera uma matriz inicial aleatória com valores entre 0 e 1
    matriz_pertinencia = matriz_pertinencia / matriz_pertinencia.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return matriz_pertinencia

### Inicialização dos medoides

É possível definir inicialmente a soma das distâncias entre os pontos como sendo:

#### $S(x_i) = \sum_{j=1}^nd(x_i,x_j)$

Sendo assim, a escolha do conjunto inicial dos medoides pode ser representada assim:

#### $M = \argmin_{x_{i1},x_{i2},...,x_{ik} \in X} \sum_{m=1}^kS(x_{im})$

Escolhemos $k$ pontos no conjunto de dados que têm a menor soma de distâncias para todos os outros pontos, minimizando assim a soma total das distâncias iniciais no algoritmo.

In [48]:
def medoids_init_fuzzy(X, U, m=2):
    n_points, n_clusters = U.shape
    print(n_clusters)
    medoids = []  # Stores selected medoids
    selected_indices = set()  # Stores indices of already chosen medoids

    # Precompute all pairwise distances to avoid recalculating
    all_distances = pairwise_distances(X)

    for c in range(n_clusters):
        U_c = U[:, c] ** m  # Weighted membership for the current cluster
        weighted_distances = np.zeros(n_points)  # Initialize weighted distances

        # Calculate weighted distances for each point as a candidate medoid
        for i in range(n_points):
            weighted_distances[i] = np.sum(U_c * all_distances[i])  # Weighted distances
        print(weighted_distances)
        # Greedy selection of the medoid with minimal weighted distance
        while True:
            medoid_index = np.argmin(weighted_distances)
            if medoid_index not in selected_indices:
                selected_indices.add(medoid_index)
                break

            weighted_distances[medoid_index] = np.inf  # Ignore this index

        medoids.append(X[medoid_index])

    return np.array(medoids)

### Atualização da matriz de pertinência

Fixo o protótipo, os graus de pertinência são atualizados com base nessa equação:

#### $u_{ik} = [\sum_{l=1}^c(\frac{d(x_k,v_i)}{d(x_k,v_l)})^{\frac{1}{m-1}}]^{-1}$

In [13]:
def atualizacao_matriz_pertinencia(dados, medoides, m):
    matriz_distancias = np.linalg.norm(dados[:, np.newaxis] - medoides, axis=2) ** 2
    matriz_distancias = np.fmax(matriz_distancias, np.finfo(np.float64).eps) # evita que matriz_distancias seja 0, np.finfo... é o menor número maior que zero aaqui
    matriz_distancias_inversa = 1 / matriz_distancias
    potencia = 1 / (m-1)
    matriz_pertinencia_atualizada = matriz_distancias_inversa ** potencia/ np.sum(matriz_distancias_inversa ** potencia, axis=1, keepdims=True) # fórmula para atualizar os graus de pertinência
    return matriz_pertinencia_atualizada

### Atualização dos medoides

Fixo os graus de pertinência, os centroides são atualizados com base nessa equação:

#### $m_i = \argmin_{p \in C_i} \sum_{x \in C_i} d(x,p)$

Essa fórmula busca, para cada medoide $m_i$, o ponto $p \in C_i$ que minimiza a soma das distâncias dentro do cluster, garantindo que o novo medoide minimize o custo de distância.

### Índice de Rand Ajustado (IRA)

In [15]:
def indice_rand(labels, predicted_labels):
    return adjusted_rand_score(labels, predicted_labels)

### Simulação de Monte Carlo

### Definição de parâmetros e execução do método

In [29]:
num_clusters = 3
num_trials = 1
m = 2
media_indice_rand, dp_indice_rand = experimento_monte_carlo_fuzzy(dados, labels, num_clusters, m, num_trials)

print(f"Monte Carlo FCMdd Clustering Results ({num_trials} trials)")
print(f"Mean Rand Index: {media_indice_rand:.4f}") # 0.7294 or 0.3589
print(f"Standard Deviation of Rand Index: {dp_indice_rand:.4f}")

Monte Carlo FCMdd Clustering Results (1 trials)
Mean Rand Index: 0.7302
Standard Deviation of Rand Index: 0.0000
